# Import

In [23]:
import sys 
import os
import configparser
import pandas as pd
import numpy as np
import datetime as dt
from faker import Faker
from sqlalchemy import create_engine

In [3]:
script_dir = os.path.abspath('')
mymodule_dir = os.path.join( script_dir, '..', '..', 'database', 'table_setup' )
sys.path.append( mymodule_dir )

import sql_queries

In [4]:
mymodule_dir = os.path.join( script_dir, '..', '..', 'database', 'connection' )
sys.path.append( mymodule_dir )

import postgres_db

# Connect

In [5]:
encoding = 'utf-16'

config_file_path = os.path.join( script_dir, '..', '..', 'database', 'connection' )
config_file_path += '/config.ini'

db_conn = postgres_db.PostgresConnector(config_file_path = config_file_path,
                                       section = 'jj-furniture',
                                       encoding = encoding)

# Data: Colors

In [6]:
color_list = ['grey', 'dark grey', 'red', 'blue', 'green', 'black', 'brown', 'beige', 'white', 'yellow']

# insert customer records

for color in color_list:
    db_conn.update(sql_queries.insert_color_table, [color])

# Data: Descriptions

In [7]:
description_list = ['sofa', 'love-seat', 'dinette set', 'mattress', 'bedframe', 'nightstand', 'chest', 'dresser', 'chair', 'desk', 'desk chair', 'bookcase', 'coffee table', 'end table'] 

# insert customer records

for description in description_list:
    db_conn.update(sql_queries.insert_description_table, [description])

# Data: Materials

In [8]:
material_list = ['fabric', 'leather', 'wood', 'metal', 'plastic']

# insert material records

for material in material_list:
    db_conn.update(sql_queries.insert_material_table, [material])

# Data: Products

In [9]:
def get_ids(table_name):
    
    query = '''
    select {}_id
    from {}
    '''
    
    results = db_conn.queryall(query.format(table_name[:-1], table_name))
    
    return results[results.columns[0]].values.tolist()

In [10]:
material_ids = get_ids('materials')
color_ids = get_ids('colors')
description_ids = get_ids('descriptions')

In [11]:
products = [
    {
     'material_id': np.random.choice(material_ids),
     'color_id': np.random.choice(color_ids),
     'description_id': np.random.choice(description_ids),
     'pieces': np.random.randint(low = 1, high = 8, size = 1)[0],
     'cost': np.round(np.random.lognormal(mean = np.log(100), sigma = 1, size = 1)[0],2) # make sure to give same cost to manufacturer 
    }
    for x in range(200)
]

In [12]:
df_products = pd.DataFrame(products)

In [13]:
# insert product records

for i, row in df_products.iterrows():
    if i % 100 == 0:
        print(i)
    db_conn.update(sql_queries.insert_product_table, 
                        (row['material_id'],
                        row['color_id'],
                        row['description_id'],
                        row['pieces'],
                        row['cost']))

0
100


# Data: Customers

In [19]:
fake = Faker()

In [ ]:
store_type = ['Store', 'Web']
num_customers = 10000

customers = [
    {
     'first_name': fake.first_name(),
     'last_name': fake.last_name(),
     'email_address': fake.email(),
     'dob': fake.date_between(start_date ='-70y', end_date = '-18y'),
     'gender': fake.profile()['sex'],
     'street_address': fake.street_address(),
     'state': fake.random_element(elements = ('NC', 'SC', 'TN', 'FL', 'GA', 'MS', 'AL')),
     'date_created': fake.date_between(start_date ='-12y'),
     'create_source': np.random.choice(store_type, p = [.8, .2]) # can give probability of selection to Store Type
    }
    for x in range(1, num_customers+1)
]

In [ ]:
df_customers = pd.DataFrame(customers)

In [ ]:
# insert customer records

for i, row in df_customers.iterrows():
    if i % num_customers*.25 == 0:
        print(i)
    db_conn.update(sql_queries.insert_customer_table, 
                        (row['first_name'],
                        row['last_name'],
                        row['email_address'],
                        row['dob'],
                        row['gender'],
                        row['street_address'],
                        row['state'],
                        row['date_created'],
                        row['create_source']))

# Data: Transactions

In [14]:
def get_cost(product):
    
    query = '''
    select cost
    from products
    where product_id = {}
    '''
    
    return float(db_conn.queryall(query.format(product))['cost'][0])

In [15]:
customer_id_list = get_ids('customers')
product_id_list = get_ids('products')

In [ ]:
num_trans = 25000
transaction_id = 1
years = list(range(2010, 2023))
weights = list(range(1, len(years) + 1))
weights = [x/sum(weights) for x in weights]

for trans in range(num_trans):
    
    if transaction_id % num_trans*.25 == 0:
        print(i)
    
    num_items = np.random.randint(low = 1, high = 5)
    year = np.random.choice(years, size = None, p = weights)
    transaction_date = fake.date_between_dates(date_start = dt.datetime.strptime(str(year) + '-01-01', '%Y-%m-%d'), date_end = dt.datetime.strptime(str(year) + '-12-31', '%Y-%m-%d'))
    customer_id = np.random.choice(customer_id_list).item()
    
    for item in range(num_items): 
        
        
        product_id = np.random.choice(product_id_list).item()
        line_item_number = item + 1
        sale_or_return = 0
        
        sale_amount = round(get_cost(product_id) * np.random.uniform(low = 1.43, high = 4), 2)
        quantity = np.random.choice([1,2,3], p = [.8, .15, .05]).item()
        
        db_conn.update(sql_queries.insert_transactions_table,
                          (transaction_id,
                          transaction_date,
                          customer_id,
                          product_id,
                          line_item_number,
                          sale_amount,
                          quantity,
                          sale_or_return))
                          
    transaction_id += 1                         

In [27]:
!jupyter nbconvert --to script *.ipynb

Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/traitlets/traitlets.py", line 656, in get
    value = obj._trait_values[self.name]
            ~~~~~~~~~~~~~~~~~^^^^^^^^^^^
KeyError: 'template_paths'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/homebrew/bin/jupyter-nbconvert", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/jupyter_core/application.py", line 285, in launch_instance
    return super().launch_instance(argv=argv, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "/opt/homebrew/lib/python3.11/site-packages/nbconvert/nbconvertapp.py", line 414, in start
    self.convert_notebooks()
  File "/opt/homebrew/lib/python3.11/site-packages/nbconvert/nbconverta